# Tutorial: Using the SpinChain class

In [108]:
# generic import statements
import numpy as np
from scipy.sparse import linalg as slinalg
from matplotlib import pyplot as plt
# import tensor product function, SpinChain class
from SpinChain import tp, SpinChain 

## Initializing a Spin Chain

First, we can initialize an empty spin chain by setting the spin, length, and boundary conditions. In this example, we initialize a chain of length `L` $=8$ with sites that contain a `spin` $=1/2$ degree of freedom. For now, we take open boundary conditions by setting `bc` to zero, but could consider periodic or antiperiodic boundary conditions with $\pm 1$.

In [100]:
chain = SpinChain(L=8, spin=1/2, bc=0) # initialize system with spin 1/2
chain_pbc = SpinChain(L=8, spin=1/2, bc=1) # periodic version of same chain
# print type and attributes
print(type(chain))
print(f'L = {chain.L}, spin = {chain.spin}, bc = {chain.bc}')

<class 'SpinChain.SpinChain'>
L = 8, spin = 0.5, bc = 0j


We also have access to the local spin operators as an attribute: `SpinChain.spin_mat_list`. In order, this returns the identity $\mathbb{1}$, the spin-z operator $S^z$, and the raising and lowering operators $S^+$ and $S^-$. A more flexible way to get local (spin) operators is to use the `SpinChain.getPauli()` method. Depending on its optional argument, this method returns the lists $\{\mathbb{1}, S^z, S^+, S^-\}$ (`basis = pm`, default), $\{\mathbb{1}, S^x, S^y, S^z\}$   (`basis = xyz`), the $2 \times 2$ Pauli matrices $\{\mathbb{1}, \sigma^x, \sigma^y, \sigma^z\}$ (`basis = pauli`), or their qudit generalization to clock and shift matrices $\{\mathbb{1}, Z, X\}$ (`basis = clock_shift`).

In [101]:
# Note that these are true spin operators (for spin-1/2 they are 1/2 of Pauli matrices)
s0, sz, sp, sm = chain.getPauli()
print(f'Sz = {sz}')
print(f'S+ = {sp}')
print(12*"*")

# exact 2x2 spin-1/2 Pauli matrics
sigma0, sigmax, sigmay, sigmaz = chain.getPauli('pauli')
print(f'\u03C3z = {sigmaz}')
print(f'\u03C3x = {sigmax}')
print(12*"*")

# Show clock and shift operators for qutrit (spin-1 -> 3-site Hilbert space)
Id, Z, X = SpinChain(spin=1, L=1, bc=0).getPauli('clock_shift')
print(f'X = {X}')

Sz = [[-0.5  0. ]
 [ 0.   0.5]]
S+ = [[0. 1.]
 [0. 0.]]
************
σz = [[ 1  0]
 [ 0 -1]]
σx = [[0 1]
 [1 0]]
************
X = [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


## Initializing a Hamiltonian

We can now make our system more interesting by including local Hamiltonian terms that define the interactions in the spin chain. This is implemented by adding the tensor product of local operators acting on a list of sites. For periodically repeating terms, one must specify a *list of local site lists* at which to apply the operators. Such local site lists should be written in *ascending order when they do not wrap around the boundary* of the chain: terms that *do* wrap around the chain are ignored for open boundary conditions.

Here, we consider a free fermion Hamiltonian with nearest-neighbor hopping in OBC, which by the [Jordan-Wigner transformation](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation) can be mapped to a nearest-neighbor spin-conserving model: $H = -t \sum_j (S^+_i S^-_{i+1} + S^-_i S^+_{i+1})$. After adding these local operators at all nearest neighbor pairs of sites, we can use `SpinChain.getHam()` to return the sparse Hamiltonian array. If needed, we can make it dense by using `numpy.ndarray.todense()`. For large systems, we can improve performance by constructing only half of the hopping terms and then calling `SpinChain.plusConj()` to automatically add the Hermitian conjugate to the Hamiltonian. 

In [102]:
t = 1
hop_left, hop_right = [sp, sm], [sm, sp] # two hopping operators
pairs = [[site, site+1] for site in range(chain.L)] # site pairs where operators act

# N.B. site indices are always taken mod L inside the SpinChain class
# This means that the last pairing in sites is interpreted as [7, 0] instead of [7, 8]
# The [7,0] coupling wraps around the 1D chain, and is only considered if bc != 0

# add nearest neighbor spin hopping to chain with obc
chain.setHamTerm(op_list = hop_left, site_list_list = pairs, coeff = -t)
chain.setHamTerm(op_list = hop_right, site_list_list = pairs, coeff = -t)

# get full sparse Hamiltonian
H = chain.getHam()
print(type(H))
print(H.shape)

<class 'scipy.sparse._arrays.bsr_array'>
(256, 256)


## Exact diagonalization

We are finally ready to actually perform the exact diagonalization. If the system is not too big ($L \lesssim 10$), the spectrum and states of the Hamiltonian can be found exactly without sparse matrix methods. Beyond that regime, we must use symmetries to reduce the size of the Hilbert space, or resort to sparse matrix methods to find a few eigenvalues/vectors.

In [120]:
# diagonalize the full 10-site spin chain
print('Full Hilbert space dimension:', chain.getHilbertDim()) # hilbert space dimensions
vals, vecs = np.linalg.eigh(H.toarray())
print(vals.shape)
print(20*'*')

# only find the lowest four energies and states using sparse matrix techniques (Lanczos)
vals_sparse, vecs_sparse = slinalg.eigsh(H, k=4, which='SA', return_eigenvectors=True)
print(vals[:4], np.sort(vals_sparse[:4]))
print(20*'*')

# project the Hamiltonian into the Magnetization = 0 sector (e.g. fixing particle number at half-filling)
# diagonalize only this projected block
chain.projectHam(M=0)
H_projected = chain.getHam()
print('Hilbert space dimension with M=0 fixed:', chain.getHilbertDim())
vals, vecs = np.linalg.eigh(H_projected.toarray())
print(vals.shape)

# return to the full Hilbert space
chain.unprojectHam()

Full Hilbert space dimension: 256
(256,)
********************
[-4.75877048 -4.41147413 -4.41147413 -4.06417777] [-4.75877048 -4.41147413 -4.41147413 -4.06417777]
********************
Hilbert space dimension with M=0 fixed: 70
(70,)
